# Tutorial: Base de datos SQLite para proyectos solares (desde Python)

**Objetivo:**  
Crear una base de datos SQLite desde Python que modele proyectos solares, sistemas por proyecto, y datos meteorológicos y eléctricos por sistema. Además veremos cómo cargar datos eléctricos desde un CSV y realizar consultas simples.

Este notebook está pensado para usar en una clase — cada celda tiene una explicación y código ejecutable.


## Requisitos

- Python 3.8+  
- `pandas` para la ingestión de CSV (instálalo si no lo tienes).

Puedes instalar pandas (si no está instalado) ejecutando la celda siguiente.

In [2]:
# Si necesitas instalar pandas, descomenta la siguiente línea:
!pip install pandas
print('Revisa si pandas está instalado; si no, instala con: pip install pandas')

Revisa si pandas está instalado; si no, instala con: pip install pandas


## Importaciones y utilidades

En esta celda definimos funciones utilitarias para conectarnos a SQLite y ajustes de rendimiento (PRAGMA).


In [3]:
import sqlite3
from datetime import datetime
from typing import Optional, Dict, Any
import os

def get_connection(db_path: str) -> sqlite3.Connection:
    conn = sqlite3.connect(db_path, detect_types=sqlite3.PARSE_DECLTYPES | sqlite3.PARSE_COLNAMES)
    conn.execute("PRAGMA foreign_keys = ON;")
    conn.execute("PRAGMA synchronous = NORMAL;")
    conn.execute("PRAGMA journal_mode = WAL;")
    return conn

print('Funciones utilitarias definidas: get_connection')

Funciones utilitarias definidas: get_connection


## Crear la estructura de la base de datos

Creamos tablas: `projects`, `systems`, `meteo_data` y `electrical_data`, más índices útiles.


In [4]:
def create_db(db_path: str):
    conn = get_connection(db_path)
    cur = conn.cursor()

    cur.execute("""
    CREATE TABLE IF NOT EXISTS projects (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT NOT NULL,
        location TEXT,
        description TEXT,
        created_at TEXT NOT NULL DEFAULT (datetime('now'))
    );
    """)

    cur.execute("""
    CREATE TABLE IF NOT EXISTS systems (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        project_id INTEGER NOT NULL,
        name TEXT NOT NULL,
        capacity_kw REAL,
        inverter_type TEXT,
        notes TEXT,
        created_at TEXT NOT NULL DEFAULT (datetime('now')),
        FOREIGN KEY (project_id) REFERENCES projects(id) ON DELETE CASCADE ON UPDATE CASCADE
    );
    """)

    cur.execute("""
    CREATE TABLE IF NOT EXISTS meteo_data (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        system_id INTEGER NOT NULL,
        timestamp TEXT NOT NULL,
        ghi REAL,
        dni REAL,
        dhi REAL,
        temp_c REAL,
        wind_m_s REAL,
        precip_mm REAL,
        source TEXT,
        inserted_at TEXT NOT NULL DEFAULT (datetime('now')),
        FOREIGN KEY (system_id) REFERENCES systems(id) ON DELETE CASCADE
    );
    """)

    cur.execute("""
    CREATE TABLE IF NOT EXISTS electrical_data (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        system_id INTEGER NOT NULL,
        timestamp TEXT NOT NULL,
        power_kw REAL,
        voltage_v REAL,
        current_a REAL,
        energy_kwh REAL,
        status TEXT,
        inserted_at TEXT NOT NULL DEFAULT (datetime('now')),
        FOREIGN KEY (system_id) REFERENCES systems(id) ON DELETE CASCADE
    );
    """)

    cur.execute("CREATE INDEX IF NOT EXISTS idx_systems_project ON systems(project_id);")
    cur.execute("CREATE INDEX IF NOT EXISTS idx_meteo_system_time ON meteo_data(system_id, timestamp);")
    cur.execute("CREATE INDEX IF NOT EXISTS idx_elec_system_time ON electrical_data(system_id, timestamp);")

    conn.commit()
    conn.close()
    print(f'BD creada en: {db_path}')

# Probar creación de DB en archivo temporal (no lo ejecutamos aun)
print('Función create_db definida')

Función create_db definida


## Funciones CRUD sencillas

Funciones para añadir proyectos y sistemas (devuelven el id insertado).


In [5]:
def add_project(db_path: str, name: str, location: Optional[str] = None, description: Optional[str] = None) -> int:
    conn = get_connection(db_path)
    cur = conn.cursor()
    cur.execute("INSERT INTO projects (name, location, description) VALUES (?, ?, ?);", (name, location, description))
    conn.commit()
    project_id = cur.lastrowid
    conn.close()
    return project_id

def add_system(db_path: str, project_id: int, name: str, capacity_kw: Optional[float] = None,
               inverter_type: Optional[str] = None, notes: Optional[str] = None) -> int:
    conn = get_connection(db_path)
    cur = conn.cursor()
    cur.execute(
        "INSERT INTO systems (project_id, name, capacity_kw, inverter_type, notes) VALUES (?, ?, ?, ?, ?);",
        (project_id, name, capacity_kw, inverter_type, notes)
    )
    conn.commit()
    system_id = cur.lastrowid
    conn.close()
    return system_id

print('Funciones add_project y add_system definidas')

Funciones add_project y add_system definidas


## Ingestión de CSV de datos eléctricos

La función siguiente usa `pandas` para leer CSV por chunks y hace inserciones por lotes. Si no tienes pandas instalado, instala con `pip install pandas`.


In [6]:
# Definimos la función pero sólo usaremos pandas si está disponible en el entorno.
try:
    import pandas as pd
except Exception:
    pd = None

def _safe_float(x):
    try:
        if x is None:
            return None
        s = str(x).strip()
        if s == '' or s.lower() in ('nan', 'none', 'null'):
            return None
        s = s.replace(',', '.')
        return float(s)
    except Exception:
        return None

def _safe_str(x):
    if x is None:
        return None
    s = str(x).strip()
    return s if s != '' else None

def ingest_electrical_csv(db_path: str, system_id: int, csv_path: str,
                          timestamp_col: str = 'timestamp',
                          col_mappings: Optional[Dict[str, str]] = None,
                          chunk_size: int = 1000):
    if pd is None:
        raise ImportError('Para usar ingest_electrical_csv necesitas instalar pandas: pip install pandas')

    if not os.path.exists(csv_path):
        raise FileNotFoundError(f'No existe el archivo CSV: {csv_path}')

    default_map = {
        'timestamp': 'timestamp',
        'time': 'timestamp',
        'datetime': 'timestamp',
        'power_kw': 'power_kw',
        'power_kW': 'power_kw',
        'power': 'power_kw',
        'voltage_v': 'voltage_v',
        'voltage': 'voltage_v',
        'current_a': 'current_a',
        'current': 'current_a',
        'energy_kwh': 'energy_kwh',
        'energy': 'energy_kwh',
        'status': 'status'
    }
    if col_mappings:
        combined_map = {**default_map, **{k: v for k, v in col_mappings.items()}}
    else:
        combined_map = default_map

    df_iter = pd.read_csv(csv_path, chunksize=chunk_size, iterator=True)

    conn = get_connection(db_path)
    cur = conn.cursor()
    total_inserted = 0
    for chunk_df in df_iter:
        chunk_df.columns = [c.strip() for c in chunk_df.columns]
        col_map_existing = {}
        for c in chunk_df.columns:
            lower = c.lower()
            if lower in combined_map:
                col_map_existing[c] = combined_map[lower]
            elif lower in {'timestamp', 'power_kw', 'voltage_v', 'current_a', 'energy_kwh', 'status'}:
                col_map_existing[c] = lower

        if not any(v == 'timestamp' for v in col_map_existing.values()):
            raise ValueError(f'El CSV no tiene una columna de timestamp reconocida. Columnas detectadas: {list(chunk_df.columns)}')

        df_to_insert = pd.DataFrame()
        for src_col, target_col in col_map_existing.items():
            df_to_insert[target_col] = chunk_df[src_col]

        df_to_insert['timestamp'] = pd.to_datetime(df_to_insert['timestamp'], infer_datetime_format=True, errors='coerce')
        df_to_insert = df_to_insert.dropna(subset=['timestamp'])
        df_to_insert['timestamp'] = df_to_insert['timestamp'].dt.strftime('%Y-%m-%d %H:%M:%S')

        for col in ['power_kw', 'voltage_v', 'current_a', 'energy_kwh', 'status']:
            if col not in df_to_insert.columns:
                df_to_insert[col] = None

        rows = [
            (system_id,
             row['timestamp'],
             _safe_float(row['power_kw']),
             _safe_float(row['voltage_v']),
             _safe_float(row['current_a']),
             _safe_float(row['energy_kwh']),
             _safe_str(row['status']))
            for _, row in df_to_insert.iterrows()
        ]

        try:
            cur.executemany(
                "INSERT INTO electrical_data (system_id, timestamp, power_kw, voltage_v, current_a, energy_kwh, status) VALUES (?, ?, ?, ?, ?, ?, ?);",
                rows
            )
            conn.commit()
            total_inserted += len(rows)
            print(f'  Insertadas {len(rows)} filas (acumulado: {total_inserted})')
        except Exception as e:
            conn.rollback()
            raise RuntimeError(f'Error insertando chunk en DB: {e}')

    conn.close()
    print(f'Ingestión terminada. Total filas insertadas: {total_inserted}')

print('Función ingest_electrical_csv definida (requiere pandas para ejecutarse)')

Función ingest_electrical_csv definida (requiere pandas para ejecutarse)


## Ejemplo: crear DB y añadir un proyecto y sistemas

Ejecuta la siguiente celda para crear la base de datos (archivo `solar_projects.db`) y añadir un proyecto con dos sistemas.


In [7]:
DB = 'solar_projects.db'
create_db(DB)

proyecto_id = add_project(DB, name='Clase - Parque Solar Demo', location='Campus', description='Proyecto demo para clase')
print('Proyecto creado con id:', proyecto_id)

sys1 = add_system(DB, proyecto_id, 'Array Este', capacity_kw=1500.0, inverter_type='String', notes='Demo Este')
sys2 = add_system(DB, proyecto_id, 'Array Oeste', capacity_kw=1000.0, inverter_type='Central', notes='Demo Oeste')
print('Sistemas creados:', sys1, sys2)

BD creada en: solar_projects.db
Proyecto creado con id: 1
Sistemas creados: 1 2


## Generar un CSV de ejemplo (datos eléctricos) y probar la ingestión

La siguiente celda crea un CSV sintético con columnas típicas usadas (timestamp, power_kW, voltage, current, energy, status).


In [8]:
import csv
from datetime import datetime, timedelta

sample_csv = 'datos_electricos_ejemplo22.csv'
start = datetime(2025, 10, 1, 0, 0)
rows = []
for i in range(24):  # 24 filas horarias de ejemplo
    t = start + timedelta(hours=i)
    power = max(0, 500 * (1 - abs(12 - i)/12))  # forma triangular simple para ejemplo
    voltage = 400 + (i % 3)  # variación pequeña
    current = power / (voltage if voltage!=0 else 1)
    energy = power / 1000.0  # kWh si fuera promedio en 1 hour
    status = 'OK' if power > 0 else 'NO_GEN'
    rows.append([t.strftime('%Y-%m-%d %H:%M:%S'), round(power,2), round(voltage,2), round(current,3), round(energy,4), status])

with open(sample_csv, 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f)
    writer.writerow(['timestamp','power_kW','voltage','current','energy','status'])
    writer.writerows(rows)

print('CSV de ejemplo creado:', sample_csv)



CSV de ejemplo creado: datos_electricos_ejemplo22.csv


## Ingestar el CSV de ejemplo

Si tienes `pandas` instalado, ejecuta la celda siguiente para cargar el CSV de ejemplo en la tabla `electrical_data` del sistema `sys1` creado antes.


In [11]:
# Ejecuta esta celda si tienes pandas instalado en tu entorno.
try:
    # aseguramos que sys1 esté definido en este scope (si el usuario re-ejecutó celdas en orden correcto)
    system_id = globals().get('sys1', None)
    if system_id is None:
        raise NameError('system_id (sys1) no encontrado. Ejecuta primero la celda que crea proyecto y sistemas.')

    # ingest (esto requiere pandas)
    ingest_electrical_csv(DB, system_id, 'datos_electricos_ejemplo22.csv')
except Exception as e:
    print('No se pudo ingestar CSV:', e)

  Insertadas 24 filas (acumulado: 24)
Ingestión terminada. Total filas insertadas: 24


C:\Users\carloscm\AppData\Local\Temp\ipykernel_14136\2867016042.py:77: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df_to_insert['timestamp'] = pd.to_datetime(df_to_insert['timestamp'], infer_datetime_format=True, errors='coerce')


## Consultar datos insertados

Consulta sencilla para mostrar las primeras filas insertadas en `electrical_data` para `sys1`.


In [12]:
import sqlite3
conn = get_connection(DB)
cur = conn.cursor()
system_id = globals().get('sys1', None)
if system_id is None:
    print('system_id no encontrado. Ejecuta la celda de creación de proyecto/sistemas.')
else:
    cur.execute('SELECT id, system_id, timestamp, power_kw, voltage_v, current_a, energy_kwh, status FROM electrical_data WHERE system_id = ? ORDER BY timestamp LIMIT 10;', (system_id,))
    rows = cur.fetchall()
    if not rows:
        print('No hay filas en electrical_data para el sistema. Quizá no ejecutaste la ingestión o pandas no está disponible.')
    else:
        for r in rows:
            print(r)
conn.close()

(1, 1, '2025-10-01 00:00:00', 0.0, 400.0, 0.0, 0.0, 'NO_GEN')
(2, 1, '2025-10-01 01:00:00', 41.67, 401.0, 0.104, 0.0417, 'OK')
(3, 1, '2025-10-01 02:00:00', 83.33, 402.0, 0.207, 0.0833, 'OK')
(4, 1, '2025-10-01 03:00:00', 125.0, 400.0, 0.312, 0.125, 'OK')
(5, 1, '2025-10-01 04:00:00', 166.67, 401.0, 0.416, 0.1667, 'OK')
(6, 1, '2025-10-01 05:00:00', 208.33, 402.0, 0.518, 0.2083, 'OK')
(7, 1, '2025-10-01 06:00:00', 250.0, 400.0, 0.625, 0.25, 'OK')
(8, 1, '2025-10-01 07:00:00', 291.67, 401.0, 0.727, 0.2917, 'OK')
(9, 1, '2025-10-01 08:00:00', 333.33, 402.0, 0.829, 0.3333, 'OK')
(10, 1, '2025-10-01 09:00:00', 375.0, 400.0, 0.938, 0.375, 'OK')


---
### Notas finales y sugerencias para la clase

- Puedes pedir a los estudiantes modificar el CSV (p.ej. cambiar nombres de columnas) y usar `col_mappings` en `ingest_electrical_csv` para mapearlos.  
- Agrega validaciones extra según necesites (rangos físicos, detección de duplicados por timestamp, etc.).  
- Si la base de datos crecerá mucho, considera usar PostgreSQL para producción; SQLite es ideal para enseñanza y prototipos.  

Si quieres, puedo:
- Generar una versión del notebook con más ejemplos (p.ej. visualización con matplotlib).  
- Añadir una CLI con `argparse` para ejecutar ingestiones desde la terminal.  
- Convertir este notebook a un archivo .py o a un PowerPoint con explicaciones.
